In [1031]:
import pandas as pd
import numpy as np
from collections import defaultdict
import pickle
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder


In [1032]:
class Loader:
    def __init__(self, path: str = "", pickled_path: str = ""):
        self.path = path
        self.pickled_path = ""
        self.df: pd.DataFrame = None

    def load(self):
        self.df = pd.read_csv(self.path)

    def activate_preprocessing(self, pre_processing_functions):
        for fun in pre_processing_functions:
            self.df = fun(self.df)

    def pickle_data(self):
        if self.df is not None:
            self.df.to_pickle(self.path + ".pickled")
            self.pickled_path = self.path + ".pickled"

    def load_pickled(self):
        self.df = pd.read_pickle(self.pickled_path)

    def get_data(self) -> pd.DataFrame:
        if self.df is not None:
            return self.df


In [1033]:
train_feats_loader = Loader(path="C:/Users/Tomer/Desktop/IML.Hackathon/Mission2_Breast_Cancer/train.feats.csv")
train_feats_loader.load()
train_feats = train_feats_loader.get_data()

<ipython-input-1032-b12d2d63caf8>:8: DtypeWarning: Columns (9,21,24) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv(self.path)


In [1034]:
train_labels_0_loader = Loader(path="C:/Users/Tomer/Desktop/IML.Hackathon/Mission2_Breast_Cancer/train.labels.0.csv")
train_labels_0_loader.load()
train_labels_0 = train_labels_0_loader.get_data()

In [1035]:
train_labels_1_loader = Loader(path="C:/Users/Tomer/Desktop/IML.Hackathon/Mission2_Breast_Cancer/train.labels.1.csv")
train_labels_1_loader.load()
train_labels_1 = train_labels_0_loader.get_data()

In [1036]:
df = train_feats.iloc[:, list(range(1,11))]

In [1037]:
## אבחנה-Ivi -Lymphovascular invasion
def lymph_invasion(df):
    df["אבחנה-Ivi -Lymphovascular invasion"].replace({
                                                    "nan": "0","none": "0","+": "1","extensive": "1","-": "0",
                                                    "No":"0", "(-)":"0","NO": "0","(+)": "1","neg": "0",
                                                    "not": "0","pos": "0","yes":"1","no":"0", None:"0",
                                                    "MICROPAPILLARY VARIANT":2}, inplace=True)
    return df


## אבחנה-KI67 protein
def ki67(df):
    non_null = df.loc[~(df["אבחנה-KI67 protein"].isnull())]
    num_only = non_null.loc[non_null["אבחנה-KI67 protein"].str.match("^\d+(?:\.\d+)?$")]
    num_only["אבחנה-KI67 protein"] = (num_only["אבחנה-KI67 protein"].astype(float))/100

    percentage_num = non_null.loc[non_null["אבחנה-KI67 protein"].str.match("^\d+(?:\.\d+)?%$")]
    percentage_num["אבחנה-KI67 protein"] = (percentage_num["אבחנה-KI67 protein"].str.replace("%","").astype(float))/100

    all_changes = pd.concat([num_only,percentage_num])
    df.update(all_changes)

    split_numbers = non_null["אבחנה-KI67 protein"].str.extract("^(\d+(?:\.\d+)?)-(\d+(?:\.\d+)?)%$").fillna(0).astype(int)
    mean_column = ((split_numbers[0]+split_numbers[1])/2)/100
    mean_column.loc[mean_column == 0] = None
    
    for index, row in df.iterrows():
        if (index in mean_column.index) and (~np.isnan(mean_column[index])):
            df.loc[index,"אבחנה-KI67 protein"] = mean_column[index]
            
    return df

## 'אבחנה-Her2'
def her2(df):
    POSITIVE_REGEX = r"(pos|Pos|POS|po|\+|[0-9]+.*[0-9]*\%*|jhuch|חיובי|strong|Strong|STRONG|high|High|HIGH|beg)"
    NEGATIVE_REGEX = r"(neg|ned|Neg|NEG|ned|nge|akhah|\-|שלילי)"

    def parse_pos_neg_col(df, col_name):
        df[col_name] = np.where(df[col_name].str.contains(POSITIVE_REGEX), "1", df[col_name])
        df[col_name] = np.where(df[col_name].str.contains(NEGATIVE_REGEX), "0", df[col_name])
    parse_pos_neg_col(df, 'אבחנה-Her2')
    
    return df


In [1038]:
# to convert to one-hot encoding
def one_hot(df):

    df.drop(df.index[df['אבחנה-Basic stage'] == 'Null'], inplace=True)
    encoder=OneHotEncoder(sparse=False)
    df_encoded = pd.DataFrame (encoder.fit_transform(df[[' Hospital','אבחנה-Basic stage','אבחנה-Histological diagnosis']]))
    df_encoded.columns = encoder.get_feature_names([' Hospital','אבחנה-Basic stage','אבחנה-Histological diagnosis'])
    df.drop([' Hospital','אבחנה-Basic stage','אבחנה-Histological diagnosis'] ,axis=1, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df_encoded.reset_index(drop=True, inplace=True)
    df= pd.concat([df, df_encoded ], axis=1)
    
    return df

# to convert to ordinal encoding
def ordinal_enc(df):
    cat = ['G1 - Well Differentiated', 'G2 - Modereately well differentiated', 'G3 - Poorly differentiated',
           'G4 - Undifferentiated','GX - Grade cannot be assessed','Null']
    ordinal_encoder = OrdinalEncoder(categories = [cat])
    df['אבחנה-Histopatological degree'] = ordinal_encoder.fit_transform(df[['אבחנה-Histopatological degree']])

    ordinal_encoder = OrdinalEncoder()
    df['User Name'] = ordinal_encoder.fit_transform(df[['User Name']])
    
    return df


In [1039]:
# fill nan, drop entries, convert values to float
def finishes(df):
    df.loc[df["אבחנה-KI67 protein"].isnull(),"אבחנה-KI67 protein"] = 0

    df = df.drop(df[~df['אבחנה-Her2'].isin([1,0,'0','1'])].index)
    df = df[pd.to_numeric(df["אבחנה-KI67 protein"], errors='coerce').notnull()]
    df["אבחנה-Ivi -Lymphovascular invasion"] = df["אבחנה-Ivi -Lymphovascular invasion"].astype(float)
    df['אבחנה-Her2'] = df['אבחנה-Her2'].astype(float)
    df["אבחנה-KI67 protein"] = df["אבחנה-KI67 protein"].astype(float)
    
    return df

In [1040]:
def tomer_prep(df):
    df = lymph_invasion(df)
    df = ki67(df)
    df = her2(df)
    df = one_hot(df)
    df = ordinal_enc(df)
    df = finishes(df)
    return df

In [1041]:
df = tomer_prep(df)
df

<ipython-input-1037-0514dc4f56af>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["אבחנה-Ivi -Lymphovascular invasion"].replace({
<ipython-input-1037-0514dc4f56af>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_only["אבחנה-KI67 protein"] = (num_only["אבחנה-KI67 protein"].astype(float))/100
<ipython-input-1037-0514dc4f56af>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,User Name,אבחנה-Age,אבחנה-Diagnosis date,אבחנה-Her2,אבחנה-Histopatological degree,אבחנה-Ivi -Lymphovascular invasion,אבחנה-KI67 protein,Hospital_20.0,Hospital_60.0,Hospital_62.0,...,אבחנה-Histological diagnosis_MUCINOUS ADENOCARCINOMA,אבחנה-Histological diagnosis_NEUROENDOCRINE CARCINOMA,אבחנה-Histological diagnosis_PAGET`S AND INTRADUCTAL CARCINOMA OF BREAST,אבחנה-Histological diagnosis_PAGET`S DISEASE OF BREAST,אבחנה-Histological diagnosis_PAPILLARY ADENOCARCINOMA,אבחנה-Histological diagnosis_PAPILLARY CARCINOMA,אבחנה-Histological diagnosis_PHYLLODES TUMOR BENIGN,אבחנה-Histological diagnosis_PHYLLODES TUMOR MALIGNANT,אבחנה-Histological diagnosis_PHYLLODES TUMOR NOS,אבחנה-Histological diagnosis_TUBULAR CARCINOMA
0,41.0,54.718724,03/10/2018 16:38,1.0,1.0,0.0,0.25,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,138.0,54.718724,03/10/2018 16:38,1.0,1.0,0.0,0.25,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,53.0,54.718724,03/10/2018 16:38,1.0,1.0,0.0,0.25,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,50.0,54.718724,03/10/2018 16:38,1.0,1.0,0.0,0.25,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,43.0,54.718724,03/10/2018 16:38,1.0,1.0,0.0,0.25,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49329,141.0,62.945237,13/01/2015 11:37,0.0,0.0,0.0,0.05,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49330,125.0,73.671108,22/08/2019 18:57,0.0,5.0,0.0,0.00,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49331,85.0,73.671108,22/08/2019 18:57,0.0,5.0,0.0,0.00,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49332,85.0,73.671108,22/08/2019 18:57,0.0,5.0,0.0,0.00,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
